In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import ipywidgets as widgets
from ipywidgets import interactive, VBox, Output
from IPython.display import display
from scipy.signal import correlate
from scipy.fft import rfft 
# (HoughLinesP ถูกย้ายไปอยู่ใน if contours)

# --- ส่วนที่ 1: ตั้งค่า ---
folder_path = r"C:\Users\Pannawit\Documents\GitHub\NARIT_Project\Calibate_Pictures\1tick"
TARGET_FILE_NAME = "1tick_1.npy"
MAX_FRAMES_TO_SHOW = 480
MATRIX_ROWS = 20
MATRIX_COLS = 20

# <<< "ช่อง" บีบกรอบ (*** แก้ไข! ***)
SHRINK_WIDTH_FACTOR = 0.9  # บีบ "กว้าง" 10%
SHRINK_HEIGHT_FACTOR = 0.9 # บีบ "สูง" 10% (เตี้ยลง)

# --- ส่วนที่ 2: โหลดไฟล์ (เหมือนเดิม) ---
file_path = os.path.join(folder_path, TARGET_FILE_NAME)
print(f"กำลังโหลดไฟล์เป้าหมาย: {file_path}")

try:
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"หาไฟล์ไม่เจอ: {file_path}")
        
    img_data = np.load(file_path)
    print(f"โหลดสำเร็จ! Shape ของข้อมูล: {img_data.shape}")

    # --- ส่วนที่ 3: *** "ผ่าตัดสมอง" (ส่วน C) *** ---
    if img_data.ndim == 4:
        
        # --- (A) "บริการ" Line Detection (Hough) (เหมือนเดิม) ---
        print("กำลังคำนวณ 'กรอบล็อก' (Hough Angle) จากเฟรมที่ 0...")
        frame_0_rgb = img_data[0].astype(np.uint8) 
        hsv = cv2.cvtColor(frame_0_rgb, cv2.COLOR_RGB2HSV) 
        lower_green = np.array([40, 50, 50])
        upper_green = np.array([80, 255, 255])
        mask = cv2.inRange(hsv, lower_green, upper_green)
        kernel = np.ones((5,5), np.uint8)
        mask_cleaned = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=3)
        contours, _ = cv2.findContours(mask_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        locked_box_points = None 
        locked_grid_points = None 
        fringe_exit_events = {}
        fringe_count_history = np.zeros(MAX_FRAMES_TO_SHOW)
        
        if contours:
            all_clean_green_points = np.concatenate(contours)
            rotated_rect_tight = cv2.minAreaRect(all_clean_green_points)
            (center_x, center_y), (width, height), blob_angle = rotated_rect_tight
            print(f"วิเคราะห์ 'ก้อน' (Blob): Center=({center_x:.0f}, {center_y:.0f}), Size=({width:.1f}, {height:.1f})")

            print("กำลังใช้ 'Line Detection' (Hough) หา 'มุม' (Angle) ของ 'ลาย' (Pattern)...")
            edges = cv2.Canny(mask_cleaned, 50, 150)
            lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 30, minLineLength=30, maxLineGap=10)
            
            if lines is None:
                raise ValueError("HoughLinesP หา 'เส้น' ไม่เจอ! (แถบไม่ชัด/ค่าสีเขียวผิด)")
                
            angles = [np.degrees(np.arctan2(line[0][3] - line[0][1], line[0][2] - line[0][0])) for line in lines]
            true_fringe_angle = np.median(angles)
            opencv_angle = true_fringe_angle - 90 
            print(f"วิเคราะห์ 'ลาย' (Hough): 'มุม' (Angle) ที่แท้จริง = {opencv_angle:.2f} องศา")
            
            # 6. "สร้าง" กรอบใหม่ + "บีบ" (*** แก้ไขส่วนสูง! ***)
            if width < height: 
                shrunk_width = width * SHRINK_HEIGHT_FACTOR # (อันนี้คือ "สูง")
                shrunk_height = height * SHRINK_WIDTH_FACTOR # (อันนี้คือ "กว้าง")
                print("สลับ W/H (เพราะ Blob 'สูง')")
            else: 
                shrunk_width = width * SHRINK_WIDTH_FACTOR
                shrunk_height = height * SHRINK_HEIGHT_FACTOR # <-- ใช้ตัวแปรใหม่
            
            shrunk_rotated_rect = ((center_x, center_y), (shrunk_width, shrunk_height), opencv_angle)
            box_points = cv2.boxPoints(shrunk_rotated_rect)
            locked_box_points = box_points.astype(int) 
            print(f"คำนวณกรอบล็อก (ขนาน & บีบ {SHRINK_HEIGHT_FACTOR*100}%) เสร็จสิ้น!")

            # --- (B) "เข็มทิศ" (Vector Correction) (เหมือนเดิม) ---
            print("กำลังคำนวณ 'เข็มทิศ' (Grid Vectors) ที่ถูกต้อง...")
            p0, p1, p2, p3 = locked_box_points 
            v1 = p1 - p0; v2 = p3 - p0 
            if np.abs(v1[0]) > np.abs(v2[0]):
                v_width = v1; v_height = v2; print("เข็มทิศชี้: v1 คือแกน U (Tracking)")
            else:
                v_width = v2; v_height = v1; print("เข็มทิศชี้: v2 คือแกน U (Tracking)")

            # 3. คำนวณ "เมทริกซ์" (เหมือนเดิม)
            print(f"กำลังคำนวณ 'เมทริกซ์ล็อก' ขนาด {MATRIX_ROWS}x{MATRIX_COLS} จุด...")
            grid_points = []
            u_coords = np.linspace(0.05, 0.95, MATRIX_COLS) 
            v_coords = np.linspace(0.05, 0.95, MATRIX_ROWS)
            for v in v_coords:
                for u in u_coords:
                    point_xy = p0 + (u * v_width) + (v * v_height) 
                    grid_points.append(point_xy.astype(int))
            locked_grid_points = grid_points 
            print("คำนวณเมทริกซ์ล็อก (ขนาน) เสร็จสิ้น!")

            # --- (C) *** "บริการ" นักสืบ (*** ผ่าตัดสมอง! ***) ---
            print("นักสืบ (FFT-Avg) กำลัง 'เดิน' ติดตามแถบมืดล่วงหน้า 480 เฟรม...")
            
            num_frames_to_process = min(img_data.shape[0], MAX_FRAMES_TO_SHOW)
            
            # *** "บริการ" ใหม่: "ตาวิเศษ" (Averaged 1D FFT Signature) ***
            
            # 1. "ดูด" Matrix 20x20 จากเฟรม 0
            gray_frame_0 = cv2.cvtColor(img_data[0].astype(np.uint8), cv2.COLOR_RGB2GRAY)
            matrix_frame_0 = np.zeros((MATRIX_ROWS, MATRIX_COLS))
            i = 0
            for r in range(MATRIX_ROWS):
                for c in range(MATRIX_COLS):
                    pt = locked_grid_points[i]
                    if 0 <= pt[1] < gray_frame_0.shape[0] and 0 <= pt[0] < gray_frame_0.shape[1]:
                        matrix_frame_0[r, c] = gray_frame_0[pt[1], pt[0]]
                    i += 1
            
            # 2. *** นี่!!! "ฉลาด" แล้ว!!! ***
            #    "เฉลี่ย" 20 แถว -> "สัญญาณสะอาด" (1x20)
            signal_frame_0_avg = np.mean(matrix_frame_0, axis=0) 
            signal_norm = signal_frame_0_avg - np.mean(signal_frame_0_avg)

            # 3. "ตาวิเศษ" (FFT) สแกน "สัญญาณสะอาด"
            fft_vals = np.abs(rfft(signal_norm))
            if len(fft_vals) < 2:
                 raise ValueError("สัญญาณ FFT ต่ำไป (ไม่เจอคลื่นในกริด)")
            dominant_freq_index = np.argmax(fft_vals[1:]) + 1
            
            # 4. "ลายเซ็น" (ที่ "ไม่มั่ว" แล้ว)
            avg_pixels_per_fringe = MATRIX_COLS / dominant_freq_index
            print(f"วิเคราะห์ (FFT-Avg): พบ {dominant_freq_index} fringe(s) ในกรอบ (แกน U)")
            print(f"==> 1 แถบมืด กว้าง {avg_pixels_per_fringe:.3f} grid points (นี่คือ 'ลายเซ็น' ใหม่)")

            # 5. "นักสืบ" เริ่มเดิน... (*** อัปเกรดให้ใช้ "ค่าเฉลี่ย" (ฉลาด) ***)
            total_pixel_shift = np.zeros(num_frames_to_process)
            last_signal_avg = signal_norm # "ลายเซ็น" (1x20) ที่สะอาดแล้ว

            for i in range(1, num_frames_to_process):
                gray_frame_i = cv2.cvtColor(img_data[i].astype(np.uint8), cv2.COLOR_RGB2GRAY)
                
                # "ดูด" Matrix 20x20 (ของเฟรม i)
                matrix_frame_i = np.zeros((MATRIX_ROWS, MATRIX_COLS))
                idx = 0
                for r in range(MATRIX_ROWS):
                    for c in range(MATRIX_COLS):
                        pt = locked_grid_points[idx]
                        if 0 <= pt[1] < gray_frame_i.shape[0] and 0 <= pt[0] < gray_frame_i.shape[1]:
                            matrix_frame_i[r, c] = gray_frame_i[pt[1], pt[0]]
                        idx += 1
                
                # "เฉลี่ย" 20 แถว (ของเฟรม i)
                current_signal_avg = np.mean(matrix_frame_i, axis=0)
                current_signal_norm = current_signal_avg - np.mean(current_signal_avg)
                
                # "เทียบหน้า" (สัญญาณสะอาด vs สัญญาณสะอาด)
                correlation = correlate(current_signal_norm, last_signal_avg, mode='same')
                pixel_shift = np.argmax(correlation) - (MATRIX_COLS // 2)
                
                if np.abs(pixel_shift) < 1: 
                    pixel_shift = 0
                
                total_pixel_shift[i] = total_pixel_shift[i-1] + pixel_shift
                last_signal_avg = current_signal_norm # อัปเดต "หน้า"
            
            # 6. แปลง "ยอดสะสม" เป็น "จำนวน Fringe" (ด้วย "ลายเซ็น" ที่ถูกต้อง)
            fringe_counts_over_time = total_pixel_shift / avg_pixels_per_fringe 
            fringe_count_history = fringe_counts_over_time 
            
            # 7. สร้าง "โพย Event" (เหมือนเดิม)
            last_fringe_num = 0
            for i in range(1, num_frames_to_process):
                fringe_num_now = int(np.floor(fringe_counts_over_time[i]))
                if fringe_num_now > last_fringe_num:
                    fringe_exit_events[i] = fringe_num_now
                    last_fringe_num = fringe_num_now 
            
            print(f"นักสืบ (FFT-Avg) จำลองการเดินเสร็จสิ้น! พบ Event การข้าม {len(fringe_exit_events)} ครั้ง")
            
        else:
            print("!!! เตือน: ไม่เจอสีเขียวในเฟรม 0 (แม้แต่ก้อนใหญ่), จะไม่วาดกรอบหรือเมทริกซ์ใดๆ")
        
        # --- (D) สร้าง Slider (ตามปกติ) ---
        num_frames = min(img_data.shape[0], MAX_FRAMES_TO_SHOW)
        frame_slider = widgets.IntSlider(value=0, min=0, max=num_frames - 1, step=1,
            description=f'{TARGET_FILE_NAME} Frame:', layout=widgets.Layout(width='80%'))

        # --- (E) สร้าง "พื้นที่ Output" (เหมือนเดิม) ---
        output_area = Output()

        # --- (F) สร้างฟังก์ชัน "อัปเดต" (เหมือนเดิมเป๊ะ) ---
        def update_on_slide(change):
            frame_idx = change['new']
            
            with output_area:
                output_area.clear_output(wait=True)
                
                # --- 1-3. ส่วนแสดงผล (*** กรอบเหลือง + จุดขาว จะ "ขนาน" และ "เตี้ยลง" ***) ---
                frame_rgb = img_data[frame_idx].astype(np.uint8)
                output_image = frame_rgb.copy()
                gray_frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2GRAY)
                
                if locked_box_points is not None:
                    # *** บรรทัดนี้... วาด "กรอบขนาน" (ที่ "บีบ" 2 แกน) ***
                    cv2.drawContours(output_image, [locked_box_points], 0, (255, 255, 0), 1)

                intensity_matrix = np.zeros((MATRIX_ROWS, MATRIX_COLS), dtype=int)

                if locked_grid_points is not None:
                    i = 0 
                    for r in range(MATRIX_ROWS):
                        for c in range(MATRIX_COLS):
                            pt_array = locked_grid_points[i]
                            pt = tuple(pt_array)
                            x, y = pt[0], pt[1]
                            # *** บรรทัดนี้... วาด "จุดขนาน" (ที่ "บีบ" 2 แกน) ***
                            cv2.circle(output_image, pt, radius=1, color=(255, 255, 255), thickness=-1)
                            if 0 <= y < gray_frame.shape[0] and 0 <= x < gray_frame.shape[1]:
                                intensity_matrix[r, c] = gray_frame[y, x]
                            i += 1 
                
                plt.figure(figsize=(8, 8)) 
                plt.imshow(output_image)
                plt.title(f"Frame {frame_idx} - Sample Points (Parallel & Shrunk)")
                plt.axis('off')
                plt.show() 

                # --- 4. พิมพ์ Matrix (*** Matrix นี้ก็จะ "ขนาน" และ "บีบ" ตาม! ***) ---
                print(f"--- Intensity Matrix (Frame {frame_idx}) ---")
                print(intensity_matrix)
                
                # --- 5. "บริการ" นักสืบ (*** ทีนี้ตัวเลขควรจะถูก! (5 fringes?) ***) ---
                print("\n--- 🕵️ บริการนักสืบ (Hough Angle + FFT-Avg Track) ---")
                current_fringe_count_decimal = fringe_count_history[frame_idx]
                print(f"ยอดสะสม (รวมถอยกลับ): {current_fringe_count_decimal:.3f} fringes")
                
                event_at_this_frame = fringe_exit_events.get(frame_idx)
                
                if event_at_this_frame:
                    print(f"!!! 🔴 EVENT: แถบมืด #{event_at_this_frame} เพิ่งข้ามเส้นชัยในเฟรมนี้ !!!")
                
                print(f"โพย Event (Frame ที่แถบใหม่ออก): {fringe_exit_events}")
                
        # --- (G) & (H) (เหมือนเดิม) ---
        frame_slider.observe(update_on_slide, names='value')
        display(VBox([frame_slider, output_area]))
        update_on_slide({'new': 0})
            
    # เคสอื่นๆ (เหมือนเดิม)
    elif img_data.ndim == 3 or img_data.ndim == 2:
        print("ไฟล์นี้เป็นภาพเดี่ยว")
        plt.imshow(img_data.squeeze())
        plt.show()
    else:
        print(f"มิติข้อมูลแปลกๆ: {img_data.shape}")

except Exception as e:
    print(f"--- เกิดข้อผิดพลาด ---")
    print(f"Error: {e}")

กำลังโหลดไฟล์เป้าหมาย: C:\Users\Pannawit\Documents\GitHub\NARIT_Project\Calibate_Pictures\1tick\1tick_1.npy
โหลดสำเร็จ! Shape ของข้อมูล: (480, 200, 200, 3)
กำลังคำนวณ 'กรอบล็อก' (Hough Angle) จากเฟรมที่ 0...
วิเคราะห์ 'ก้อน' (Blob): Center=(95, 88), Size=(152.6, 221.5)
กำลังใช้ 'Line Detection' (Hough) หา 'มุม' (Angle) ของ 'ลาย' (Pattern)...
วิเคราะห์ 'ลาย' (Hough): 'มุม' (Angle) ที่แท้จริง = -1.22 องศา
สลับ W/H (เพราะ Blob 'สูง')
คำนวณกรอบล็อก (ขนาน & บีบ 90.0%) เสร็จสิ้น!
กำลังคำนวณ 'เข็มทิศ' (Grid Vectors) ที่ถูกต้อง...
เข็มทิศชี้: v2 คือแกน U (Tracking)
กำลังคำนวณ 'เมทริกซ์ล็อก' ขนาด 20x20 จุด...
คำนวณเมทริกซ์ล็อก (ขนาน) เสร็จสิ้น!
นักสืบ (FFT-Avg) กำลัง 'เดิน' ติดตามแถบมืดล่วงหน้า 480 เฟรม...
วิเคราะห์ (FFT-Avg): พบ 8 fringe(s) ในกรอบ (แกน U)
==> 1 แถบมืด กว้าง 2.500 grid points (นี่คือ 'ลายเซ็น' ใหม่)
นักสืบ (FFT-Avg) จำลองการเดินเสร็จสิ้น! พบ Event การข้าม 6 ครั้ง


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import ipywidgets as widgets
from ipywidgets import interactive, VBox, Output
from IPython.display import display
# <<< "สมอง" ใหม่... (ไล่ 'correlate' ออก!)
from scipy.signal import butter, filtfilt 

# --- ส่วนที่ 1: ตั้งค่า ---
folder_path = r"C:\Users\Pannawit\Documents\GitHub\NARIT_Project\Calibate_Pictures\1tick"
TARGET_FILE_NAME = "1tick_1.npy"
MAX_FRAMES_TO_SHOW = 480

# <<< "ช่อง" แก้ไข: "ถี่ขึ้น" + "เตี้ยลง"
MATRIX_ROWS = 40 # (สำหรับ "โชว์" Matrix)
MATRIX_COLS = 40 # (สำหรับ "โชว์" Matrix)
SHRINK_WIDTH_FACTOR = 0.9  # บีบ "กว้าง" 10%
SHRINK_HEIGHT_FACTOR = 0.85 # บีบ "สูง" 15% (เตี้ยลง)

# <<< "ช่อง" พารามิเตอร์ (60 fps!)
FRAME_RATE = 60.0 # <--- คุณบอก 60fps!
LOWPASS_CUTOFF_HZ = 5.0 # กรอง "Noise" ที่เร็วกว่า 5 Hz (เช่น การสั่น 60fps)

# --- ส่วนที่ 2: โหลดไฟล์ (เหมือนเดิม) ---
file_path = os.path.join(folder_path, TARGET_FILE_NAME)
print(f"กำลังโหลดไฟล์เป้าหมาย: {file_path}")

try:
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"หาไฟล์ไม่เจอ: {file_path}")
        
    img_data = np.load(file_path)
    print(f"โหลดสำเร็จ! Shape ของข้อมูล: {img_data.shape}")

    # --- ส่วนที่ 3: *** "Cyclops" (All-Pixel Phase Unwrapping) *** ---
    if img_data.ndim == 4:
        
        # --- (A) "บริการ" Line Detection (Hough) (เหมือนเดิม... "ตาดี") ---
        print("กำลังคำนวณ 'กรอบล็อก' (Hough Angle) จากเฟรมที่ 0...")
        frame_0_rgb = img_data[0].astype(np.uint8) 
        hsv = cv2.cvtColor(frame_0_rgb, cv2.COLOR_RGB2HSV) 
        lower_green = np.array([40, 50, 50])
        upper_green = np.array([80, 255, 255])
        mask = cv2.inRange(hsv, lower_green, upper_green)
        kernel = np.ones((5,5), np.uint8)
        mask_cleaned = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=3)
        contours, _ = cv2.findContours(mask_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        locked_box_points = None 
        locked_grid_points_display = None # <--- "กริด" (สำหรับ "โชว์")
        locked_pixel_coords = None      # <--- "All Pixels" (สำหรับ "คิด")
        
        fringe_exit_events = {}
        fringe_count_history = np.zeros(MAX_FRAMES_TO_SHOW)
        
        if contours:
            all_clean_green_points = np.concatenate(contours)
            rotated_rect_tight = cv2.minAreaRect(all_clean_green_points)
            (center_x, center_y), (width, height), blob_angle = rotated_rect_tight
            print(f"วิเคราะห์ 'ก้อน' (Blob): Center=({center_x:.0f}, {center_y:.0f}), Size=({width:.1f}, {height:.1f})")

            print("กำลังใช้ 'Line Detection' (Hough) หา 'มุม' (Angle) ของ 'ลาย' (Pattern)...")
            edges = cv2.Canny(mask_cleaned, 50, 150)
            lines = cv2.HoughLinesP(edges, 1, np.pi / 180, 30, minLineLength=30, maxLineGap=10)
            
            if lines is None:
                raise ValueError("HoughLinesP หา 'เส้น' ไม่เจอ! (แถบไม่ชัด/ค่าสีเขียวผิด)")
                
            angles = [np.degrees(np.arctan2(line[0][3] - line[0][1], line[0][2] - line[0][0])) for line in lines]
            true_fringe_angle = np.median(angles)
            opencv_angle = true_fringe_angle - 90 
            print(f"วิเคราะห์ 'ลาย' (Hough): 'มุม' (Angle) ที่แท้จริง = {opencv_angle:.2f} องศา")
            
            # 6. "สร้าง" กรอบใหม่ + "บีบ" (*** 0.85 ***)
            if width < height: 
                shrunk_width = width * SHRINK_HEIGHT_FACTOR 
                shrunk_height = height * SHRINK_WIDTH_FACTOR 
            else: 
                shrunk_width = width * SHRINK_WIDTH_FACTOR
                shrunk_height = height * SHRINK_HEIGHT_FACTOR 
            
            shrunk_rotated_rect = ((center_x, center_y), (shrunk_width, shrunk_height), opencv_angle)
            box_points = cv2.boxPoints(shrunk_rotated_rect)
            locked_box_points = box_points.astype(int) 
            print(f"คำนวณกรอบล็อก (ขนาน & บีบ H:{SHRINK_HEIGHT_FACTOR*100}%) เสร็จสิ้น!")

            # --- (B) *** "บริการ" ใหม่: "เอา grid ออก" (All-Pixel Mask) *** ---
            print("กำลังคำนวณ 'พิกเซลทั้งหมด' (All Pixels) ภายในกรอบ...")
            
            # 1. สร้าง "Mask" (สีดำ)
            mask_roi = np.zeros_like(mask_cleaned)
            # 2. "วาด" (Fill) กรอบเหลือง (ที่ "ขนาน" และ "บีบ" แล้ว)
            cv2.drawContours(mask_roi, [locked_box_points], 0, 1, -1) # -1 = Fill
            
            # 3. "ดูด" พิกเซล (y, x) "ทั้งหมด" ที่อยู่ข้างใน (เช่น 5000+ จุด)
            #    (นี่คือ "All Pixels" ที่คุณขอ!)
            locked_pixel_coords = np.argwhere(mask_roi == 1) # (N, 2) array
            print(f"คำนวณ 'All Pixels' เสร็จสิ้น! พบ {len(locked_pixel_coords)} พิกเซล")

            # --- (B.2) *** สร้าง "กริด" (สำหรับ "โชว์" ... ไม่ได้ใช้ "คิด") *** ---
            print(f"กำลังคำนวณ 'เมทริกซ์โชว์' ขนาด {MATRIX_ROWS}x{MATRIX_COLS} จุด...")
            p0, p1, p2, p3 = locked_box_points 
            v1 = p1 - p0; v2 = p3 - p0 
            if np.abs(v1[0]) > np.abs(v2[0]): v_width = v1; v_height = v2
            else: v_width = v2; v_height = v1
            grid_points = []
            u_coords = np.linspace(0.05, 0.95, MATRIX_COLS); v_coords = np.linspace(0.05, 0.95, MATRIX_ROWS)
            for v in v_coords:
                for u in u_coords:
                    point_xy = p0 + (u * v_width) + (v * v_height) 
                    grid_points.append(point_xy.astype(int))
            locked_grid_points_display = grid_points # <--- (อันนี้ไว้ "โชว์" เฉยๆ)
            
            # --- (C) *** "บริการ" Cyclops (ใช้ "All Pixels") *** ---
            print("Cyclops (All-Pixel) กำลัง 'ประมวลผล' 480 เฟรม...")
            
            num_frames_to_process = min(img_data.shape[0], MAX_FRAMES_TO_SHOW)
            
            # 1. *** "ดูด" "ภาพรวม" (จาก "All Pixels") ***
            mean_intensity_signal = np.zeros(num_frames_to_process)
            
            # (แยก 'y' กับ 'x' ออกมา... เร็วกว่า)
            all_y = locked_pixel_coords[:, 0]
            all_x = locked_pixel_coords[:, 1]
            
            for i in range(num_frames_to_process):
                gray_frame_i = cv2.cvtColor(img_data[i].astype(np.uint8), cv2.COLOR_RGB2GRAY)
                
                # "ดูด" "ทุกจุด" (5000+ จุด) ใน "All Pixels"
                current_intensities = gray_frame_i[all_y, all_x]
                
                # "เฉลี่ย" (Average) มัน!
                mean_intensity_signal[i] = np.mean(current_intensities)
            
            # 2. *** "บริการ" ใหม่: "กรอง" (Filter) Noise 60fps (ที่คุณบอก!) ***
            nyquist_freq = FRAME_RATE / 2.0
            normalized_cutoff = LOWPASS_CUTOFF_HZ / nyquist_freq
            b, a = butter(4, normalized_cutoff, btype='low')
            filtered_signal = filtfilt(b, a, mean_intensity_signal) # <--- "สัญญาณสะอาด"
            print(f"กรอง (Filter) Noise 60fps (ตัด > {LOWPASS_CUTOFF_HZ} Hz) เสร็จสิ้น!")

            # 3. "ตาวิเศษ" (Phase Unwrapping) (จาก "สัญญาณสะอาด")
            I_norm = filtered_signal - np.mean(filtered_signal)
            Q_norm = np.gradient(I_norm)
            phase_history = np.arctan2(Q_norm, I_norm)
            unwrapped_phase = np.unwrap(phase_history)
            
            # 4. แปลง "Radian" เป็น "Fringes"
            fringe_counts_over_time = (unwrapped_phase - unwrapped_phase[0]) / (2 * np.pi)
            fringe_counts_over_time = np.abs(fringe_counts_over_time) # (กลับเครื่องหมาย)
            fringe_count_history = fringe_counts_over_time 
            
            # 5. *** "บริการ" ใหม่: "โพย Event" (Hysteresis - กันนับซ้ำ!) ***
            last_fringe_num = 0
            for i in range(1, num_frames_to_process):
                # (เช็ค "สัญญาณสะอาด")
                f_now_int = int(np.floor(fringe_counts_over_time[i])) 
                
                # (ถ้า "เลขจำนวนเต็ม" มัน "เพิ่มขึ้น" เท่านั้น!)
                if f_now_int > last_fringe_num:
                    fringe_exit_events[i] = f_now_int # จด Event! {152: 1}
                    last_fringe_num = f_now_int # อัปเดตตัวนับ (มันจะ "รอ" จนกว่าจะข้าม 2.0)
            
            print(f"Cyclops (All-Pixel) ประมวลผลเสร็จสิ้น! พบ Event การข้าม {len(fringe_exit_events)} ครั้ง")
            
        else:
            print("!!! เตือน: ไม่เจอสีเขียวในเฟรม 0 (แม้แต่ก้อนใหญ่), จะไม่วาดกรอบหรือเมทริกซ์ใดๆ")
        
        # --- (D) สร้าง Slider (ตามปกติ) ---
        num_frames = min(img_data.shape[0], MAX_FRAMES_TO_SHOW)
        frame_slider = widgets.IntSlider(value=0, min=0, max=num_frames - 1, step=1,
            description=f'{TARGET_FILE_NAME} Frame:', layout=widgets.Layout(width='80%'))

        # --- (E) สร้าง "พื้นที่ Output" (เหมือนเดิม) ---
        output_area = Output()

        # --- (F) สร้างฟังก์ชัน "อัปเดต" (*** UI เหมือนเดิม! ***) ---
        def update_on_slide(change):
            frame_idx = change['new']
            
            with output_area:
                output_area.clear_output(wait=True)
                
                # --- 1-3. ส่วนแสดงผล (*** "กริด" 40x40 (สำหรับ "โชว์") ***) ---
                frame_rgb = img_data[frame_idx].astype(np.uint8)
                output_image = frame_rgb.copy()
                gray_frame = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2GRAY)
                
                if locked_box_points is not None:
                    cv2.drawContours(output_image, [locked_box_points], 0, (255, 255, 0), 1)

                intensity_matrix = np.zeros((MATRIX_ROWS, MATRIX_COLS), dtype=int)

                if locked_grid_points_display is not None: # <--- (ใช้ "กริดโชว์")
                    i = 0 
                    for r in range(MATRIX_ROWS):
                        for c in range(MATRIX_COLS):
                            pt_array = locked_grid_points_display[i] # <--- (ใช้ "กริดโชว์")
                            pt = tuple(pt_array)
                            x, y = pt[0], pt[1]
                            cv2.circle(output_image, pt, radius=1, color=(255, 255, 255), thickness=-1)
                            if 0 <= y < gray_frame.shape[0] and 0 <= x < gray_frame.shape[1]:
                                intensity_matrix[r, c] = gray_frame[y, x]
                            i += 1 
                
                plt.figure(figsize=(8, 8)) 
                plt.imshow(output_image)
                plt.title(f"Frame {frame_idx} - Sample Points (Parallel & Shrunk)")
                plt.axis('off')
                plt.show() 

                # --- 4. พิมพ์ Matrix (*** 40x40 (สำหรับ "โชว์") ***) ---
                print(f"--- Intensity Matrix (Frame {frame_idx}) ---")
                print(intensity_matrix)
                
                # --- 5. "บริการ" Cyclops (*** ทีนี้ตัวเลขควรจะถูก! (5 fringes?) ***) ---
                print("\n--- 👁️ บริการ Cyclops (All-Pixel + Hysteresis) ---")
                current_fringe_count_decimal = fringe_count_history[frame_idx]
                print(f"ยอดสะสม (รวมถอยกลับ): {current_fringe_count_decimal:.3f} fringes")
                
                event_at_this_frame = fringe_exit_events.get(frame_idx)
                
                if event_at_this_frame:
                    print(f"!!! 🔴 EVENT: แถบมืด #{event_at_this_frame} เพิ่งข้ามเส้นชัยในเฟรมนี้ !!!")
                
                print(f"โพย Event (Frame ที่แถบใหม่ออก): {fringe_exit_events}")
                
        # --- (G) & (H) (เหมือนเดิม) ---
        frame_slider.observe(update_on_slide, names='value')
        display(VBox([frame_slider, output_area]))
        update_on_slide({'new': 0})
            
    # เคสอื่นๆ (เหมือนเดิม)
    elif img_data.ndim == 3 or img_data.ndim == 2:
        print("ไฟล์นี้เป็นภาพเดี่ยว")
        plt.imshow(img_data.squeeze())
        plt.show()
    else:
        print(f"มิติข้อมูลแปลกๆ: {img_data.shape}")

except Exception as e:
    print(f"--- เกิดข้อผิดพลาด ---")
    print(f"Error: {e}")

กำลังโหลดไฟล์เป้าหมาย: C:\Users\Pannawit\Documents\GitHub\NARIT_Project\Calibate_Pictures\1tick\1tick_1.npy
โหลดสำเร็จ! Shape ของข้อมูล: (480, 200, 200, 3)
กำลังคำนวณ 'กรอบล็อก' (Hough Angle) จากเฟรมที่ 0...
วิเคราะห์ 'ก้อน' (Blob): Center=(95, 88), Size=(152.6, 221.5)
กำลังใช้ 'Line Detection' (Hough) หา 'มุม' (Angle) ของ 'ลาย' (Pattern)...
วิเคราะห์ 'ลาย' (Hough): 'มุม' (Angle) ที่แท้จริง = -1.22 องศา
คำนวณกรอบล็อก (ขนาน & บีบ H:85.0%) เสร็จสิ้น!
กำลังคำนวณ 'พิกเซลทั้งหมด' (All Pixels) ภายในกรอบ...
คำนวณ 'All Pixels' เสร็จสิ้น! พบ 24569 พิกเซล
กำลังคำนวณ 'เมทริกซ์โชว์' ขนาด 40x40 จุด...
Cyclops (All-Pixel) กำลัง 'ประมวลผล' 480 เฟรม...
กรอง (Filter) Noise 60fps (ตัด > 5.0 Hz) เสร็จสิ้น!
Cyclops (All-Pixel) ประมวลผลเสร็จสิ้น! พบ Event การข้าม 6 ครั้ง


In [4]:
import numpy as np
import cv2
from skimage.transform import hough_line, hough_line_peaks, rotate
from skimage.feature import canny
from skimage.color import rgb2gray
from scipy.signal import hilbert
import sys
import os # เพิ่ม os เพื่อเช็ค path

def load_data(filepath):
    """
    โหลดไฟล์ .npy และตรวจสอบข้อมูลเบื้องต้น
    """
    print(f"กำลังโหลดไฟล์: {filepath}")
    if not os.path.exists(filepath):
        print(f"ข้อผิดพลาด: ไม่พบไฟล์ที่ '{filepath}'")
        print("กรุณาตรวจสอบ Path หรือใช้ r'...' (raw string) สำหรับ Windows")
        return None
        
    try:
        frames = np.load(filepath)
        if frames.dtype != np.uint8:
            print(f"Warning: Data type is {frames.dtype}, not uint8.")
            if np.max(frames) <= 1.0:
                print("ข้อมูลเป็น float (0-1), กำลังแปลงเป็น uint8 (0-255)...")
                frames = (frames * 255).astype(np.uint8)
        
        print(f"โหลดไฟล์สำเร็จ! Shape: {frames.shape}")
        return frames
    except Exception as e:
        print(f"ข้อผิดพลาดในการโหลดไฟล์: {e}")
        return None

def get_fringe_angle(frame):
    """
    ใช้ Hough Transform เพื่อหามุมเอียงของลายแทรกสอดในเฟรมแรก
    """
    print("กำลังคำนวณมุมเอียงของลาย (Hough Transform)...")
    gray = rgb2gray(frame)
    
    # เพิ่ม sigma ช่วยให้ Canny เห็นเส้นเบลอๆ ได้ดีขึ้น
    edges = canny(gray, sigma=3.0) 
    
    # ทดสอบมุมในช่วงแคบๆ รอบแนวตั้ง
    tested_angles = np.deg2rad(np.linspace(-15, 15, 180))
    h, theta, d = hough_line(edges, theta=tested_angles)
    
    _, angles, _ = hough_line_peaks(h, theta, d)
    
    if len(angles) == 0:
        print("Warning: ไม่พบเส้นที่ชัดเจน, จะใช้มุม 0 (แนวตั้ง) เป็นค่าเริ่มต้น")
        return 0.0
    
    avg_angle_rad = np.mean(angles)
    print(f"ตรวจพบมุมเอียง: {np.rad2deg(avg_angle_rad):.2f} องศา (จากแนวตั้ง)")
    return avg_angle_rad

def get_tilted_line_points(angle_rad, shape):
    """
    คำนวณจุดเริ่มต้น (pt1) และจุดสิ้นสุด (pt2) ของเส้นเอียง
    ที่พาดผ่านจุดศูนย์กลาง (cx, cy)
    """
    h, w = shape[0], shape[1]
    cx, cy = w // 2, h // 2 # จุดศูนย์กลาง (100, 100)

    # เราต้องการเส้นที่ 'ขนาน' กับลาย
    # มุมจาก hough (theta) คือมุมของ 'normal' (ตั้งฉาก)
    # มุมของเส้น (phi) คือ theta + 90 องศา
    # แต่เนื่องจากลายเกือบตั้ง, theta เกือบ 0, phi จะเกือบ 90
    
    # วิธีที่ง่ายกว่า: ใช้มุม theta ที่ได้มา (ซึ่งเกือบ 0)
    # แล้วคำนวณจุดตัดที่ขอบบน (y=0) และขอบล่าง (y=h-1)
    
    cos_theta = np.cos(angle_rad)
    sin_theta = np.sin(angle_rad)
    
    # rho คือระยะห่างจาก (0,0) ไปยังเส้น
    rho = cx * cos_theta + cy * sin_theta

    # คำนวณ x-intercepts (ที่ y=0 และ y=h-1)
    # x = (rho - y*sin(theta)) / cos(theta)
    
    # ป้องกันการหารด้วย 0 (ถ้าเส้นตั้งตรงเป๊ะๆ cos(theta) อาจจะน้อยมาก)
    if np.abs(cos_theta) < 1e-6:
        x0 = cx
        x1 = cx
    else:
        x0 = (rho - 0 * sin_theta) / cos_theta
        x1 = (rho - (h - 1) * sin_theta) / cos_theta

    pt1 = (int(x0), 0)
    pt2 = (int(x1), h - 1)
    
    return pt1, pt2

def calculate_fringe_count_via_phase(frames):
    """
    ใช้ Hilbert Transform ในการนับเฟส (และจำนวนแถบ) จากพิกเซลเดียว
    """
    print("กำลังวิเคราะห์เฟสที่จุดอ้างอิง (y=100, x=100)...")
    
    # 1. ดึงสัญญาณ time-series จากจุดกลาง (ใช้ช่องสีเขียว (index 1))
    # (480,)
    time_series = frames[:, 100, 100, 1].astype(float)

    # 2. ลบ mean (Demean) เพื่อให้สัญญาณแกว่งรอบ 0
    signal = time_series - np.mean(time_series)

    # 3. ใช้ Hilbert Transform
    analytic_signal = hilbert(signal)

    # 4. หามุม (เฟส) และทำการ Unwrap (นี่คือตัวแก้ Hysteresis)
    instantaneous_phase = np.unwrap(np.angle(analytic_signal))

    # 5. คำนวณจำนวนแถบ
    # เฟสที่เปลี่ยนไป 2*pi คือ 1 แถบ
    # การเคลื่อนที่ R -> L ทำให้ค่าสีที่ (100,100) เปลี่ยน (เช่น ดำ->เขียว->ดำ)
    # เราต้องดูว่าเฟสเพิ่มหรือลด
    
    # (เราจะใส่เครื่องหมายลบไว้ก่อน ถ้ามันนับถอยหลัง ค่อยสลับเป็นบวก)
    # (อัปเดต: จากภาพที่เลื่อน R->L เฟสมักจะลดลง, - (ลบ) = บวก)
    count_over_time = -(instantaneous_phase - instantaneous_phase[0]) / (2 * np.pi)
    
    print("วิเคราะห์เฟสเสร็จสิ้น!")
    return count_over_time

def main():
    # !!! แก้ไข Path ตรงนี้ให้ถูกต้อง !!!
    # (ใช้ r"..." (raw string) เพื่อป้องกันปัญหา backslash ใน Windows)
    filepath = r"C:\Users\Pannawit\Documents\GitHub\NARIT_Project\Calibate_Pictures\1tick\1tick_2.npy"
    
    frames = load_data(filepath)
    if frames is None:
        return

    num_frames, h, w = frames.shape[0], frames.shape[1], frames.shape[2]

    # --- 1. หาเส้นเอียง (จากเฟรมแรก) ---
    angle_rad = get_fringe_angle(frames[0])
    pt1, pt2 = get_tilted_line_points(angle_rad, (h, w))
    
    print(f"สร้างเส้นอ้างอิงเอียง จาก {pt1} ไป {pt2}")

    # --- 2. คำนวณการนับ (จากทุกเฟรม) ---
    # เราส่ง frames ทั้งหมดเข้าไปคำนวณทีเดียวเลย
    count_over_time = calculate_fringe_count_via_phase(frames)
    
    print("--- เริ่มแสดงผลวิดีโอ (กด 'q' เพื่อออก) ---")
    
    # --- 3. แสดงผล (60 fps) ---
    # 1000ms / 60fps = 16.67ms -> ใช้ waitKey(16) หรือ 17
    wait_time_ms = int(1000 / 60) # ประมาณ 16ms
    
    for i in range(num_frames):
        display_frame = frames[i].copy()
        
        current_count = count_over_time[i]
        
        # วาดเส้นเอียง (สีเหลือง)
        cv2.line(display_frame, pt1, pt2, (0, 255, 255), 1) # BGR = Yellow
        
        # วาดข้อความแสดงผล
        text_frame = f"Frame: {i}/{num_frames}"
        cv2.putText(display_frame, text_frame, (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        text_count = f"Count: {current_count:.3f}"
        cv2.putText(display_frame, text_count, (10, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 100), 2) # สีเขียว
        
        cv2.imshow("Fringe Counter (60 fps)", display_frame)
        
        # รอ 16ms และเช็คว่ากด 'q' หรือไม่
        if cv2.waitKey(wait_time_ms) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()
    
    print("========================================")
    print("--- 👁️ ผลการนับด้วย Phase Analysis ---")
    print(f"ยอดสะสมสุดท้าย: {count_over_time[-1]:.3f} fringes")
    print("========================================")

if __name__ == "__main__":
    # ตรวจสอบว่ามี Libraries ครบ
    # pip install opencv-python scikit-image scipy
    main()

กำลังโหลดไฟล์: C:\Users\Pannawit\Documents\GitHub\NARIT_Project\Calibate_Pictures\1tick\1tick_2.npy
โหลดไฟล์สำเร็จ! Shape: (480, 200, 200, 3)
กำลังคำนวณมุมเอียงของลาย (Hough Transform)...
สร้างเส้นอ้างอิงเอียง จาก (100, 0) ไป (100, 199)
กำลังวิเคราะห์เฟสที่จุดอ้างอิง (y=100, x=100)...
วิเคราะห์เฟสเสร็จสิ้น!
--- เริ่มแสดงผลวิดีโอ (กด 'q' เพื่อออก) ---
--- 👁️ ผลการนับด้วย Phase Analysis ---
ยอดสะสมสุดท้าย: -9.901 fringes
